# 비지도학습 감성분석 - Lexicon 기반

In [61]:
import pandas as pd
import numpy as np
from google.colab import files
up = files.upload()

Saving labeledTrainData.tsv to labeledTrainData.tsv


In [62]:
import warnings
warnings.filterwarnings('ignore')

### Wordnet Synset 및 Sentiwordnet sentisynset

In [63]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [64]:
from nltk.corpus import wordnet

In [65]:
term = 'fly'
synsets = wordnet.synsets(term)
type(synsets), len(synsets)

(list, 20)

In [66]:
for synset in synsets:
  print(f'#### name: {synset.name()} ####')
  print('POS:', synset.lexname())
  print('정의:', synset.definition())
  print('표제어:', synset.lemma_names())

#### name: fly.n.01 ####
POS: noun.animal
정의: two-winged insects characterized by active flight
표제어: ['fly']
#### name: tent-fly.n.01 ####
POS: noun.artifact
정의: flap consisting of a piece of canvas that can be drawn back to provide entrance to a tent
표제어: ['tent-fly', 'rainfly', 'fly_sheet', 'fly', 'tent_flap']
#### name: fly.n.03 ####
POS: noun.artifact
정의: an opening in a garment that is closed by a zipper or by buttons concealed under a fold of cloth
표제어: ['fly', 'fly_front']
#### name: fly.n.04 ####
POS: noun.act
정의: (baseball) a hit that flies up in the air
표제어: ['fly', 'fly_ball']
#### name: fly.n.05 ####
POS: noun.artifact
정의: fisherman's lure consisting of a fishhook decorated to look like an insect
표제어: ['fly']
#### name: fly.v.01 ####
POS: verb.motion
정의: travel through the air; be airborne
표제어: ['fly', 'wing']
#### name: fly.v.02 ####
POS: verb.motion
정의: move quickly or suddenly
표제어: ['fly']
#### name: fly.v.03 ####
POS: verb.motion
정의: operate an airplane
표제어: ['fly', 'av

 - 어휘 간 유사도

In [67]:
# 단어, 품사를 모를 때 쓰는 방법 (synset()으로 알아내는 방법)
for synset in wordnet.synsets('tiger'):
  print(synset.name(), synset.definition())

tiger.n.01 a fierce or audacious person
tiger.n.02 large feline of forests in most of Asia having a tawny coat with black stripes; endangered


In [68]:
# 단어, 품사를 알 때 쓰는 방법 => synset() 바로 쓴다.
tiger = wordnet.synset('tiger.n.02')
tree = wordnet.synset('tree.n.01')
lion = wordnet.synset('lion.n.01')
cat = wordnet.synset('cat.n.01')
dog = wordnet.synset('dog.n.01')

In [69]:
# 단어 간의 유사도
tiger.path_similarity(lion), tiger.path_similarity(tree), tiger.path_similarity(dog), tiger.path_similarity(cat)

(0.3333333333333333, 0.07142857142857142, 0.16666666666666666, 0.25)

In [70]:
# 5개 단어간의 유사도
similarities = []
entities = [tree, lion, tiger, cat, dog]
for entity in entities:
    similarity = [entity.path_similarity(another) for another in entities]
    similarities.append(similarity)

In [71]:
df = pd.DataFrame(similarities, columns=['tree', 'lion', 'tiger', 'cat', 'dog'], index = ['tree', 'lion', 'tiger', 'cat', 'dog'])
df

,tree,lion,tiger,cat,dog
tree,1.000000,0.071429,0.071429,0.076923,0.125000
lion,0.071429,1.000000,0.333333,0.250000,0.166667
tiger,0.071429,0.333333,1.000000,0.250000,0.166667
cat,0.076923,0.250000,0.250000,1.000000,0.200000
dog,0.125000,0.166667,0.166667,0.200000,1.000000


In [72]:
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [73]:
from nltk.corpus import sentiwordnet

In [74]:
senti_synsets = list(sentiwordnet.senti_synsets('slow'))
senti_synsets

[SentiSynset('decelerate.v.01'),
 SentiSynset('slow.v.02'),
 SentiSynset('slow.v.03'),
 SentiSynset('slow.a.01'),
 SentiSynset('slow.a.02'),
 SentiSynset('dense.s.04'),
 SentiSynset('slow.a.04'),
 SentiSynset('boring.s.01'),
 SentiSynset('dull.s.08'),
 SentiSynset('slowly.r.01'),
 SentiSynset('behind.r.03')]

In [75]:
# father 단어의 긍정/부정/객관성 지수
father = sentiwordnet.senti_synset('father.n.01')
father.pos_score(), father.neg_score(), father.obj_score()

(0.0, 0.0, 1.0)

In [76]:
# mother 단어의 긍정/부정/객관성 지수
mother = sentiwordnet.senti_synset('mother.n.01')
mother.pos_score(), mother.neg_score(), mother.obj_score()

(0.0, 0.0, 1.0)

In [77]:
# love 단어의 긍정/부정/객관성 지수
love = sentiwordnet.senti_synset('love.v.01')
love.pos_score(), love.neg_score(), love.obj_score()

(0.5, 0.0, 0.5)

In [78]:
# fabulous 단어의 긍정/부정/객관성 지수
fabulous = sentiwordnet.senti_synset('fabulous.a.01')
fabulous.pos_score(), fabulous.neg_score(), fabulous.obj_score()

(0.875, 0.125, 0.0)

In [79]:
wordnet.NOUN, wordnet.ADJ, wordnet.ADV, wordnet.VERB

('n', 'a', 'r', 'v')

# 감성지수 계산

In [80]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [81]:
from nltk import word_tokenize, pos_tag
sentence = "It's good to see you again."
word_lst = word_tokenize(sentence)
word_lst

['It', "'s", 'good', 'to', 'see', 'you', 'again', '.']

In [82]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [83]:
pos_tag(word_lst)

[('It', 'PRP'),
 ("'s", 'VBZ'),
 ('good', 'JJ'),
 ('to', 'TO'),
 ('see', 'VB'),
 ('you', 'PRP'),
 ('again', 'RB'),
 ('.', '.')]

In [84]:
def penn_to_wordnet(tag):
    
    if tag.startswith('N'):
        
        return wordnet.NOUN

    if tag.startswith('J'):
        
        return wordnet.ADJ

    if tag.startswith('R'):
        
        return wordnet.ADV

    if tag.startswith('V'):
        
        return wordnet.VERB

In [85]:
for word, pos in pos_tag(word_lst):
    print(word, penn_to_wordnet(pos))

It None
's v
good a
to None
see v
you None
again r
. None


- Sentence로부터 Senti_synset 객체로 만드는 과정

In [86]:
sentence = "It's good to see you again."
word_lst = [word for word in word_tokenize(sentence) if len(word) > 2]
word_lst

['good', 'see', 'you', 'again']

In [87]:
for word, pos in pos_tag(word_lst):
    wn_tag = penn_to_wordnet(pos)
    if wn_tag:              # None이 아닌 'n','a','r','v'
        synsets = list(sentiwordnet.senti_synsets(word, wn_tag))
        synset = synsets[0]
        print(synset)

<good.a.01: PosScore=0.75 NegScore=0.0>
<see.n.01: PosScore=0.0 NegScore=0.0>
<again.r.01: PosScore=0.0 NegScore=0.0>


In [ ]:
sentiment = 0
for word, pos in pos_tag(word_lst):
    wn_tag = penn_to_wordnet(pos)
   
    if wn_tag: # none이 아닌 'a', 'r', 'v', 'n'
        synsets = list(sentiwordnet.senti_synsets(word, wn_tag))
        synset = synsets[0]
        sentiment += synset.pos_score() - synset.neg_score()

sentiment

In [40]:
from nltk import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [89]:
# 표제어 추출까지 고려
sentiment = 0

for word, pos in pos_tag(word_lst):
    wn_tag = penn_to_wordnet(pos)

    if wn_tag:              # None이 아닌 'n','a','r','v'
        lemma = lemmatizer.lemmatize(word, wn_tag)
        synsets = list(sentiwordnet.senti_synsets(lemma, wn_tag))
        synset = synsets[0]
        sentiment += synset.pos_score() - synset.neg_score()

sentiment

0.75

- Document에서 감성 지수를 계산하는 과정 및 함수

In [90]:
from nltk import sent_tokenize
dc = '''Everything about this movie is trying too hard - the over dramatic score, the long shots on characters faces, the overacting, the complex crime story - it all feels it's trying to get an Oscar in every moment.
It's overly long, drawn out, and the story feels like a generic crime saga that has the Batman universe shoehorned into it.
This movie is not a masterpiece, but it spends a lot of effort making you think it is!'''

In [92]:
# 감성지수 계산하는 과정
sentiment = 0.0
for sentence in sent_tokenize(dc): # 문장을 토큰으로 쪼개는 과정
    word_list = [word for word in word_tokenize(sentence) if len(word) > 2]

    for word, pos in pos_tag(word_list):
        wn_tag = penn_to_wordnet(pos)

        if wn_tag:              # None이 아닌 'n','a','r','v'
            lemma = lemmatizer.lemmatize(word, wn_tag)
            synsets = list(sentiwordnet.senti_synsets(lemma, wn_tag))

            if not synsets:
                print(word)
                continue

            synset = synsets[0]
            sentiment += synset.pos_score() - synset.neg_score()  # 긍정인지 부정인지 점수를 계산하는 과정

print('긍정' if sentiment >= 0 else '부정')

Everything
shots
masterpiece
lot
긍정


In [93]:
# 위의 코드를 함수화
def swn_polarity(text):
    lemmatizer = WordNetLemmatizer()
    sentiment = 0.0

    for sentence in sent_tokenize(text):
        word_list = [word for word in word_tokenize(sentence) if len(word) > 2]
        
        for word, pos in pos_tag(word_list):
            wn_tag = penn_to_wordnet(pos)
           
            if wn_tag:              # None이 아닌 'n','a','r','v'
                lemma = lemmatizer.lemmatize(word, wn_tag)
                synsets = list(sentiwordnet.senti_synsets(lemma, wn_tag))
               
                if not synsets:
                    continue
                
                synset = synsets[0]
                sentiment += synset.pos_score() - synset.neg_score()

    return 1 if sentiment >= 0 else 0

 - IMDB 영화평 감성분석

In [100]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t', quoting=3)   # 3: QUOTE_NONE
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [101]:
# <br /> 태그는 공백으로 변환
df.review = df.review.str.replace('<br />',' ')

In [102]:
# 구둣점, 숫자 제거 - 영문자가 아닌 글자는 공백으로 변환
df.review = df.review.str.replace('[^A-Za-z]', ' ').str.strip()

In [103]:
df.shape

(25000, 3)

In [104]:
df = df.head(10000)

In [105]:
%time df['pred'] = df.review.apply(lambda x: swn_polarity(x))

CPU times: user 3min 25s, sys: 1.13 s, total: 3min 26s
Wall time: 3min 31s


In [106]:
# 정확도 계산
from sklearn.metrics import accuracy_score
accuracy_score(df.sentiment, df.pred)

0.6309

### VADER Lexicon 이용한 감성 분석

In [107]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [109]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
senti_analyzer = SentimentIntensityAnalyzer()
senti_analyzer.polarity_scores(df.review[0])

{'compound': -0.7943, 'neg': 0.13, 'neu': 0.743, 'pos': 0.127}

In [110]:
def vader_polarity(dc, threshold=0.1):
    score = senti_analyzer.polarity_scores(dc)
    return 1 if score['compound'] >= threshold else 0

In [111]:
%time df['vader'] = df.review.apply(lambda x: vader_polarity(x, 0.1))

CPU times: user 29.7 s, sys: 643 ms, total: 30.4 s
Wall time: 30.4 s


In [112]:
accuracy_score(df.sentiment, df.vader)

0.6997